# Agenți AI Azure cu suport pentru Protocolul de Context al Modelului (MCP) - Python

Acest notebook demonstrează cum să utilizați agenții AI Azure cu instrumente Protocolul de Context al Modelului (MCP) în Python. Arată cum să creați un agent inteligent care poate folosi servere MCP externe (cum ar fi Microsoft Learn) pentru capabilități îmbunătățite, utilizând autentificarea fără cheie.


## Instalarea Pachetelor Python Necesare

Mai întâi, trebuie să instalăm pachetele Python necesare:
- **azure-ai-projects**: SDK-ul principal pentru proiectele Azure AI
- **azure-ai-agents**: SDK-ul Azure AI Agents pentru crearea și gestionarea agenților
- **azure-identity**: Oferă autentificare fără chei folosind DefaultAzureCredential
- **mcp**: Implementarea Protocolului Contextului Modelului pentru Python


## Beneficiile Autentificării Fără Chei

Acest notebook demonstrează **autentificarea fără chei**, care oferă mai multe avantaje:
- ✅ **Fără chei API de gestionat** - Utilizează autentificarea bazată pe identitatea Azure
- ✅ **Securitate îmbunătățită** - Fără secrete stocate în cod sau fișiere de configurare
- ✅ **Rotirea automată a acreditivelor** - Azure gestionează ciclul de viață al acreditivelor
- ✅ **Control al accesului bazat pe roluri** - Utilizează Azure RBAC pentru permisiuni detaliate
- ✅ **Suport pentru medii multiple** - Funcționează fără probleme atât în dezvoltare, cât și în producție

`DefaultAzureCredential` selectează automat cea mai bună sursă de acreditive disponibilă:
1. **Identitate gestionată** (când rulează în Azure)
2. Acreditivele **Azure CLI** (în timpul dezvoltării locale)
3. Acreditivele **Visual Studio**
4. **Variabile de mediu** (dacă sunt configurate)
5. Autentificarea prin **browser interactiv** (ca soluție de rezervă)


## Configurarea Autentificării Fără Cheie

**Cerințe preliminare pentru autentificarea fără cheie:**

### Pentru Dezvoltare Locală:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Pentru Medii Azure:
- Activează **Identitatea Gestionată Atribuită Sistemului** pe resursa ta Azure
- Atribuie rolurile **RBAC** corespunzătoare identității gestionate:
  - `Cognitive Services OpenAI User` pentru acces la Azure OpenAI
  - `AI Developer` pentru acces la Proiectele Azure AI

### Variabile de Mediu (Opțional):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Nu sunt necesare chei API sau stringuri de conexiune!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importați Bibliotecile Necesare

Importați modulele Python necesare:
- **os, time**: Biblioteci standard Python pentru variabile de mediu și întârzieri
- **AIProjectClient**: Clientul principal pentru Proiectele AI Azure
- **DefaultAzureCredential**: Autentificare fără cheie pentru serviciile Azure
- **Clasele legate de MCP**: Pentru crearea și gestionarea instrumentelor MCP și manipularea aprobărilor


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Configurați Setările Serverului MCP

Configurați setările serverului MCP utilizând variabile de mediu cu valori implicite de rezervă:
- **MCP_SERVER_URL**: URL-ul serverului MCP (implicit este API-ul Microsoft Learn)
- **MCP_SERVER_LABEL**: O etichetă pentru identificarea serverului MCP (implicit este "mslearn")

Această abordare permite o configurare flexibilă în diferite medii.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Crearea unui Client pentru Proiectul Azure AI (Autentificare fără cheie)

Inițializați clientul pentru Proiectul Azure AI utilizând **autentificarea fără cheie**:
- **endpoint**: URL-ul endpoint-ului proiectului Azure AI Foundry
- **credential**: Folosește `DefaultAzureCredential()` pentru o autentificare sigură, fără cheie
- **Nu sunt necesare chei API**: Descoperă și utilizează automat cele mai bune acreditive disponibile

**Fluxul de autentificare:**
1. Verifică pentru Identitate Gestionată (în medii Azure)
2. Revine la acreditivele Azure CLI (pentru dezvoltare locală)
3. Utilizează alte surse de acreditive disponibile, după caz

Această abordare elimină necesitatea gestionării cheilor API sau a stringurilor de conexiune în codul dvs.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Creare Definiție Instrument MCP

Creează un instrument MCP care se conectează la serverul MCP Microsoft Learn:
- **server_label**: Identificator pentru serverul MCP
- **server_url**: Punctul final URL al serverului MCP
- **allowed_tools**: Listă opțională pentru a restricționa ce instrumente pot fi utilizate (lista goală permite toate instrumentele)

Acest instrument va permite agentului să acceseze documentația și resursele Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Creare Agent și Executare Conversație (Flux de lucru fără chei)

Această secțiune cuprinzătoare demonstrează întregul **flux de lucru al agentului fără chei**:

1. **Creare Agent AI**: Configurează un agent cu modelul GPT-4.1 nano și instrumente MCP
2. **Creare Fir de Discuție**: Stabilește un fir de conversație pentru comunicare
3. **Trimitere Mesaj**: Întreabă agentul despre diferențele dintre Azure OpenAI și OpenAI
4. **Gestionare Aprobări pentru Instrumente**: Aprobă automat apelurile instrumentelor MCP atunci când este necesar
5. **Monitorizare Execuție**: Urmărește progresul agentului și gestionează orice acțiuni necesare
6. **Afișare Rezultate**: Arată detaliile conversației și utilizării instrumentelor

**Caracteristici fără chei:**
- ✅ **Fără secrete hardcodate** - Toată autentificarea este gestionată prin identitatea Azure
- ✅ **Securizat implicit** - Utilizează controlul accesului bazat pe roluri
- ✅ **Implementare simplificată** - Nu este necesară gestionarea acreditivelor
- ✅ **Prietenos cu auditul** - Toate accesările sunt urmărite prin identitatea Azure

Agentul va folosi instrumentele MCP pentru a accesa resursele Microsoft Learn cu securitate completă și fără gestionarea cheilor API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
